In [1]:
import streamlit as sl
import pandas as pd

#read data
df = pd.read_csv('https://raw.githubusercontent.com/dvdmenu/streamlit_demo_app/main/P4-Movie-Ratings.csv')

df.head()

sl.title("Title")
sl.write("Titles are written with 'sl.title('your text')', this is written in 'sl.write()' '")

#this will create a scrollable window for the dataframe
sl.write(df)



sl.bar_chart(data=df, x='Audience Ratings %', y='Budget (million $)', use_container_width=True)

2023-11-13 13:21:42.372 
  command:

    streamlit run C:\Users\Lenovo\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Film                       559 non-null    object
 1   Genre                      559 non-null    object
 2   Rotten Tomatoes Ratings %  559 non-null    int64 
 3   Audience Ratings %         559 non-null    int64 
 4   Budget (million $)         559 non-null    int64 
 5   Year of release            559 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 26.3+ KB


In [6]:
df['Year of release'].describe()

count     559.000000
mean     2009.152057
std         1.362632
min      2007.000000
25%      2008.000000
50%      2009.000000
75%      2010.000000
max      2011.000000
Name: Year of release, dtype: float64

## Data Cleaning: Inflation data

In [21]:
# Retrieved inflation from consumer price index, filtered by USA.
# base period 2010 = 100. This means that if 2011 CPI equals 115, prices have gone up 15 % from 2010. 
# https://databank.worldbank.org/reports.aspx?source=2&series=FP.CPI.TOTL&country=USA#

In [9]:
infl = pd.read_excel('inflation_yearly.xlsx')
infl.head()

,Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Consumer price index (2010 = 100),FP.CPI.TOTL,United States,USA,13.563061,13.708284,13.872615,14.04459,14.224207,14.449685,...,106.833849,108.566932,108.695722,110.067009,112.411557,115.157303,117.244195,118.690502,124.266414,134.211206
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
drop_these = [1,2,3,4,5]
infl = infl.drop(index=drop_these)

In [24]:
drop_columns = ['Series Name', 'Country Name', 'Country Code', 'Series Code']
infl = infl.drop(columns=drop_columns)

In [25]:
melted_infl = pd.melt(infl, var_name='Year', value_name='InflationRate')
melted_infl

,Year,InflationRate
0,1960 [YR1960],13.563061
1,1961 [YR1961],13.708284
2,1962 [YR1962],13.872615
3,1963 [YR1963],14.044590
4,1964 [YR1964],14.224207
...,...,...
58,2018 [YR2018],115.157303
59,2019 [YR2019],117.244195
60,2020 [YR2020],118.690502
61,2021 [YR2021],124.266414


In [34]:
melted_infl['Year'] = melted_infl['Year'].str.extract(r'(\d{4})').astype(int)

In [36]:
melted_infl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           63 non-null     int32  
 1   InflationRate  63 non-null     float64
dtypes: float64(1), int32(1)
memory usage: 888.0 bytes


 ## Feature engineering: Adding inflation corrected budgets to dataframe

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Film                       559 non-null    object
 1   Genre                      559 non-null    object
 2   Rotten Tomatoes Ratings %  559 non-null    int64 
 3   Audience Ratings %         559 non-null    int64 
 4   Budget (million $)         559 non-null    int64 
 5   Year of release            559 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 26.3+ KB


In [46]:
# add yearly inflation rate to dataframe.
# there might be a more elegant solution than to have the yearly inflation rate on every row, but I am following a procedure from Maven Analytics' Data Prep & EDA course 

In [39]:
df = pd.merge(df, melted_infl, left_on='Year of release', right_on='Year', how='inner')

In [43]:
df.drop('Year', axis=1, inplace=True)

In [44]:
df

,Film,Genre,Rotten Tomatoes Ratings %,Audience Ratings %,Budget (million $),Year of release,InflationRate
0,(500) Days of Summer,Comedy,87,81,8,2009,98.386420
1,12 Rounds,Action,30,52,20,2009,98.386420
2,17 Again,Comedy,55,70,20,2009,98.386420
3,2012,Action,39,63,200,2009,98.386420
4,A Serious Man,Drama,89,64,7,2009,98.386420
...,...,...,...,...,...,...,...
554,We Bought a Zoo,Comedy,63,79,50,2011,103.156842
555,What's Your Number,Comedy,24,50,20,2011,103.156842
556,X-Men: First Class,Action,87,88,160,2011,103.156842
557,Your Highness,Comedy,26,36,50,2011,103.156842
